In [1]:
'''
CITIES THAT WERE DELETED:
- NEW ZEALAND
- ROME
- LISBON
- IRELAND
- LOS ANGELES
- MALAGA
'''

'\nCITIES THAT WERE DELETED:\n- NEW ZEALAND\n- ROME\n- LISBON\n- IRELAND\n- LOS ANGELES\n- MALAGA\n'

In [2]:
import pandas as pd
import numpy as np

Make a new data frame in which for each listing, the availabilites are calculated for each month in the data frame

In [3]:
# read the file for the location (for now, use Amsterdam)
calendar = pd.read_csv("home_rentals\Amsterdam\calendar.csv.gz")

# make a new column which only has month-year 
calendar["date"] = pd.to_datetime(calendar["date"])
calendar["month"] = calendar["date"].dt.month
calendar["year"] = calendar["date"].dt.year
calendar['month_year'] = calendar['month'].astype(str) + '-' + calendar['year'].astype(str)

# make the new data frame
available_by_month = (calendar[calendar['available'] == 't'].groupby(['listing_id', 'month_year']).size().unstack(fill_value=0).stack().reset_index(name='days_available'))
available_by_month



,listing_id,month_year,days_available
0,2818,1-2023,0
1,2818,10-2022,8
2,2818,11-2022,6
3,2818,12-2022,0
4,2818,2-2023,0
...,...,...,...
58677,709986109543860789,6-2023,0
58678,709986109543860789,7-2023,0
58679,709986109543860789,8-2023,0
58680,709986109543860789,9-2022,20


In [4]:
# now we have to calculate a single average for each month between all the listings, so as to get an average availability 
# for a standard property in the location for each month

# Calculate the average days_available for each month_year (making sure that it rounds to full numbers)
average_by_month = (available_by_month.groupby('month_year')['days_available'].mean().round(0).reset_index(name='average_days_available'))

# convert the month_year column to datetime format 
average_by_month['month_year'] = pd.to_datetime(average_by_month['month_year'], format='%m-%Y').dt.to_period('M')

# sort the rows by ascending dates
average_by_month = average_by_month.sort_values('month_year')

# reset the row index
average_by_month.reset_index(drop=True, inplace=True)

average_by_month



,month_year,average_days_available
0,2022-09,4.0
1,2022-10,8.0
2,2022-11,12.0
3,2022-12,12.0
4,2023-01,14.0
5,2023-02,13.0
6,2023-03,12.0
7,2023-04,10.0
8,2023-05,12.0
9,2023-06,11.0


In [6]:
# now make a data frame with the average number of rentals per month per location



In [5]:
''' 

TO DO:

- split the data set into test and train 
- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september
    then it's skewed data because it'll say it was either completely free/completely booked for that month)


'''

" \n\nTO DO:\n\n- split the data set into test and train \n- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september\n    then it's skewed data because it'll say it was either completely free/completely booked for that month)\n\n\n"